## Human Game Play

In [38]:
import torch.nn as nn
import numpy as np
from random import randint
import torch

In [39]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(42, 500)
        self.relu1 = nn.LeakyReLU()
        self.linear2 = nn.Linear(500, 1000)
        self.relu2 = nn.LeakyReLU()
        self.linear3 = nn.Linear(1000, 3)
        self.drop1   = nn.Dropout(0.15)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.drop1(x)
        x = self.linear3(x)
        return x

In [40]:
net = torch.load('9.pkl').cuda()

In [41]:
# score = []
# gameboard = np.zeros([6,7]).reshape(-1).astype('float32')
# print(gameboard.shape)
# gameboard = torch.from_numpy(gameboard).cuda()
# output = net(gameboard)

In [42]:
# output.tolist().index(max(output))

### Game

In [43]:
class Ai_Game_Connect_four():
    def __init__(self):
        # computer  1
        # player   -1
        # blank     0
        self.board = np.zeros([6,7])
        self.endgame = 0
        self.ComputerTurn = True
        self.player = 'X'
        self.computer = 'O'
        
        
    def printBoard(self):
#         print(self.board)
        pboard = [[' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ']]
        
        
        
        for i in range(6):
            for j in range(7):
                if self.board[i][j] == 0:
                    pboard[i][j] = " "
                elif self.board[i][j] == 1:
                    pboard[i][j] = self.computer
                else:
                    pboard[i][j] = self.player
                    
        rows = ['a','b','c','d','e','f']
        tops = '    1   2   3   4   5   6   7 '
        row = [[n] for n in range(0,7)]
        row[0][0] = 'f | '
        row[1][0] = 'e | '
        row[2][0] = 'd | '
        row[3][0] = 'c | '
        row[4][0] = 'b | '
        row[5][0] = 'a | '
        print('')
        print('  ' + '-'*(len(tops)-3))
        for j in range(0,len(rows)):
            for i in range(1,8):
                row[j][0] = row[j][0] + str(pboard[j][i-1]) + ' | '
            print(row[j][0])
            print('  ' + '-'*((len(row[j][0])-3)))
        print(tops)
        print('')
        
        
    def move(self):
        AccMove = {'1','2','3','4','5','6','7'}
        
        if self.ComputerTurn == True:
            Ai_choice = [0,1,2,3,4,5,6]
            All_choice_win = []
            broad_list = []
            for choice in Ai_choice:
                if self.board[0][choice] != 0:
                    pass
                else:
                    test_map = np.array(self.board)
                    for num in range(len(test_map.T[0])):

                        if test_map.T[choice][num] != 0:
                            test_map.T[choice][num-1] = 1
                            break
                        elif num == 5:
                            test_map.T[choice][5] = 1       
                        broad_list.append(test_map)
                        
                    test_map = test_map.reshape(-1).astype('float32')
                    test_map = torch.from_numpy(test_map).cuda()
                    output = net(test_map.cuda()).tolist()
                    All_choice_win.append(output[0])
                    
            # here we choose the hihghest win probability as AI's next step.
            #So we load this model on our game. Where the AI ​​plays next depends on where the chess is played to determine the current highest victory. 
            num = All_choice_win.index(max(All_choice_win))
            print(num)
            
            pos = int(num)
            for num in range(len(game.board.T[0])):
                if self.board.T[pos-1][num] != 0:
                    self.board.T[pos-1][num-1] = 1
                    return
                elif num == 5:
                    self.board.T[pos-1][5] = 1
       
            
        else:
            pos = str(input('Your move: '))
            if (pos in AccMove) == False:
                print('Input must be integer between 1 and 7')
                self.move()   
            elif self.board[0][int(pos)-1] != 0:
                print("FULL")
                self.move()
            else:
                pos = int(pos)
                for num in range(len(game.board.T[0])):
                    if self.board.T[pos-1][num] != 0:
                        self.board.T[pos-1][num-1] = -1
                        return
                    elif num == 5:
                        self.board.T[pos-1][5] = -1

                        
    def Check_if_Done(self):
        done = [False,'No Winner Yet']
        #horizontal check
        for i in range(6):
            for j in range(4):
                if self.board[i][j] == self.board[i][j+1] == self.board[i][j+2] == self.board[i][j+3] == -1:
                    done = [True,'You Wins Horizontal']
                if self.board[i][j] == self.board[i][j+1] == self.board[i][j+2] == self.board[i][j+3] == 1:
                    done = [True,'Computer Wins Horizontal']
        #vertical check
        for j in range(7):
            for i in range(3):
                if self.board[i][j] == self.board[i+1][j] == self.board[i+2][j] == self.board[i+3][j] == -1:
                    done = [True,'You Wins Vertical']
                if self.board[i][j] == self.board[i+1][j] == self.board[i+2][j] == self.board[i+3][j] == 1:
                    done = [True,'Computer Wins Vertical']
        #diagonal check top left to bottom right
        for row in range(3):
            for col in range(4):
                if self.board[row][col] == self.board[row + 1][col + 1] == self.board[row + 2][col + 2] == self.board[row + 3][col + 3] == -1:
                    done = [True,'You Wins Diagonal']
                if self.board[row][col] == self.board[row + 1][col + 1] == self.board[row + 2][col + 2] == self.board[row + 3][col + 3] == 1:
                    done = [True,'Computer Wins Diagonal']

        #diagonal check bottom left to top right
        for row in range(5, 2, -1):
            for col in range(3):
                if self.board[row][col] == self.board[row - 1][col + 1] == self.board[row - 2][col + 2] == self.board[row - 3][col + 3] == -1:
                    done = [True,'You Wins Diagonal']
                if self.board[row][col] == self.board[row - 1][col + 1] == self.board[row - 2][col + 2] == self.board[row - 3][col + 3] == 1:
                    done = [True,'Computer Wins Diagonal']
        return done
    
    def main(self):
        # Prompt player input
        
        
        print("X First, O Second")
        self.player = str( input('Choose X or O: ') )
        if self.player != 'X' and self.player != 'O':
            self.player = str( input('Choose X or O: ') )
        if self.player == 'X':
            self.computer = 'O'
        else:
            self.computer = 'X'        
        
        self.printBoard()
        print('To play: enter an integer between 1 to 7 ' + \
          'corresponding to each column in the board. ' + \
          'Whoever stacks 4 pieces next to each other, ' + \
          'either horizontally, vertically or diagonally wins.')
        
        GameOver = False
        if self.player == 'X':
            self.ComputerTurn = False
        else:
            self.ComputerTurn = True

        while GameOver == False:
            if self.Check_if_Done()[0]:
                GameOver = True
                print(self.Check_if_Done()[1])
                break
            else:
                pass
            self.move()
            self.ComputerTurn = not self.ComputerTurn
            self.printBoard()

        return

In [47]:
game = Ai_Game_Connect_four()
game.main()

X First, O Second


Choose X or O:  O



  ---------------------------
f |   |   |   |   |   |   |   | 
  -----------------------------
e |   |   |   |   |   |   |   | 
  -----------------------------
d |   |   |   |   |   |   |   | 
  -----------------------------
c |   |   |   |   |   |   |   | 
  -----------------------------
b |   |   |   |   |   |   |   | 
  -----------------------------
a |   |   |   |   |   |   |   | 
  -----------------------------
    1   2   3   4   5   6   7 

To play: enter an integer between 1 to 7 corresponding to each column in the board. Whoever stacks 4 pieces next to each other, either horizontally, vertically or diagonally wins.
3

  ---------------------------
f |   |   |   |   |   |   |   | 
  -----------------------------
e |   |   |   |   |   |   |   | 
  -----------------------------
d |   |   |   |   |   |   |   | 
  -----------------------------
c |   |   |   |   |   |   |   | 
  -----------------------------
b |   |   |   |   |   |   |   | 
  -----------------------------
a |   |  

Your move:  2



  ---------------------------
f |   |   |   |   |   |   |   | 
  -----------------------------
e |   |   |   |   |   |   |   | 
  -----------------------------
d |   |   |   |   |   |   |   | 
  -----------------------------
c |   |   |   |   |   |   |   | 
  -----------------------------
b |   |   |   |   |   |   |   | 
  -----------------------------
a |   | O | X |   |   |   |   | 
  -----------------------------
    1   2   3   4   5   6   7 

3

  ---------------------------
f |   |   |   |   |   |   |   | 
  -----------------------------
e |   |   |   |   |   |   |   | 
  -----------------------------
d |   |   |   |   |   |   |   | 
  -----------------------------
c |   |   |   |   |   |   |   | 
  -----------------------------
b |   |   | X |   |   |   |   | 
  -----------------------------
a |   | O | X |   |   |   |   | 
  -----------------------------
    1   2   3   4   5   6   7 



...


  ---------------------------
f |   |   |   |   | O |   |   | 
  -----------------------------
e |   |   |   |   | X |   |   | 
  -----------------------------
d |   |   |   |   | O |   | O | 
  -----------------------------
c |   | O | O |   | X |   | O | 
  -----------------------------
b |   | O | X |   | O |   | X | 
  -----------------------------
a | X | O | X | X | X |   | X | 
  -----------------------------
    1   2   3   4   5   6   7 

4

  ---------------------------
f |   |   |   |   | O |   |   | 
  -----------------------------
e |   |   |   |   | X |   |   | 
  -----------------------------
d |   |   |   |   | O |   | O | 
  -----------------------------
c |   | O | O |   | X |   | O | 
  -----------------------------
b |   | O | X | X | O |   | X | 
  -----------------------------
a | X | O | X | X | X |   | X | 
  -----------------------------
    1   2   3   4   5   6   7 



Your move:  1



  ---------------------------
f |   |   |   |   | O |   |   | 
  -----------------------------
e |   |   |   |   | X |   |   | 
  -----------------------------
d |   |   |   |   | O |   | O | 
  -----------------------------
c |   | O | O |   | X |   | O | 
  -----------------------------
b | O | O | X | X | O |   | X | 
  -----------------------------
a | X | O | X | X | X |   | X | 
  -----------------------------
    1   2   3   4   5   6   7 

0

  ---------------------------
f |   |   |   |   | O |   |   | 
  -----------------------------
e |   |   |   |   | X |   | X | 
  -----------------------------
d |   |   |   |   | O |   | O | 
  -----------------------------
c |   | O | O |   | X |   | O | 
  -----------------------------
b | O | O | X | X | O |   | X | 
  -----------------------------
a | X | O | X | X | X |   | X | 
  -----------------------------
    1   2   3   4   5   6   7 



Your move:  2



  ---------------------------
f |   |   |   |   | O |   |   | 
  -----------------------------
e |   |   |   |   | X |   | X | 
  -----------------------------
d |   | O |   |   | O |   | O | 
  -----------------------------
c |   | O | O |   | X |   | O | 
  -----------------------------
b | O | O | X | X | O |   | X | 
  -----------------------------
a | X | O | X | X | X |   | X | 
  -----------------------------
    1   2   3   4   5   6   7 

You Wins Vertical
